# Hands on Example - Simulating Bond

Attention: Change variable "simulated years" in "simulate_bond.py"/"simulate_multi_cty_bond.py" to 7 before running the notebook

In [50]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
import simulate_multi_cty_bond as smcb
import simulate_bond as sb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define Variables

In [51]:
#define nominals
nominal_sng = 200

#define premium and risk free rate
premium = 0.05
rf_rate = 0.00

#number countries
countries = [1,2]

#specify payouts and damages per event
pay_dam_df_c1 = pd.DataFrame([{'pay': np.array(140.0), 'damage': np.array(150.0), 'year': 0, 'month': np.array(2)},
                              {'pay': np.array(50.0), 'damage': np.array(50.0), 'year': 5, 'month': np.array(3)}])

pay_dam_df_c2 = pd.DataFrame([{'pay': np.array(40.0), 'damage': np.array(35.0), 'year': 1, 'month': np.array(10)},
                              {'pay': np.array(160.0), 'damage': np.array(200.0), 'year': 5, 'month': np.array(9)}])

pay_dam_df_dic = {1: pay_dam_df_c1,
                  2: pay_dam_df_c2}

nominal_dic = {1: nominal_sng,
               2: nominal_sng}

## Simulate Single-Country Bonds

### Country 01

In [69]:
exp_loss_pool_c1, att_prob_pool_c1, df_loss_month_c1, es_metrics_c1 = sb.init_exp_loss_att_prob_simulation(pay_dam_df_c1, nominal_sng)
metrics_pool_c1, returns_pool_c1 = sb.init_bond_simulation(pay_dam_df_c1, premium, rf_rate, nominal_sng)

Expected Loss = 0.07916666666666666
Attachment Probability = 0.16666666666666666


In [70]:
metrics_pool_c1

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments,Summed Payments,Total Premiums
0,0.05,-0.185713,-0.351349,0.95,-10.0,95.0,190.0,98.291667


### Country 02

In [76]:
exp_loss_pool_c2, att_prob_pool_c2, df_loss_month_c1, es_metrics_c2 = sb.init_exp_loss_att_prob_simulation(pay_dam_df_c2, nominal_sng)
metrics_pool_c2, returns_pool_c2 = sb.init_bond_simulation(pay_dam_df_c2, premium, rf_rate, nominal_sng, ann_com)

Expected Loss = 0.10000000000000002
Attachment Probability = 0.25


In [65]:
metrics_pool_c2

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments,Summed Payments,Total Premiums
0,0.05,-0.237994,-0.533473,0.888889,-30.0,80.0,240.0,111.333333


## Simulate Multi-Country Bond

In [108]:
import functions as fct
from pathlib import Path
exp_loss_pool, att_prob_pool, df_loss_month, total_losses, es_metrics, MES_cty = smcb.init_exp_loss_att_prob_simulation(countries, pay_dam_df_dic, nominal_sng, nominal_dic)
tranches = fct.create_tranches(np.array([2]), total_losses, df_loss_month, Path("C:/Users/Kai/OneDrive - UT Cloud/Documents/Studium_ETH/Thesis"))
ncf, prem_cty_df = smcb.simulate_ncf_prem(premium, df_loss_month, tranches, MES_cty, 'regression')

Expected Loss = 0.17499999999999996
Attachment Probability = 0.3333333333333333


c:\Users\Kai\OneDrive - UT Cloud\Documents\Studium_ETH\Thesis\Code\master_thesis_cat_bonds\functions.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tranches = pd.concat([tranch_df, pd.DataFrame(rows)], ignore_index=True)


In [122]:
print(f"Sharpe Ratio: {np.mean(ncf['Total'])/np.std(ncf['Total'])}")

Sharpe Ratio: -0.4270180530470342


In [124]:
print(f"Coverage: {es_metrics['Payout']/es_metrics['Damage']}")

Coverage: 0.8936170212765957
